**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install sk-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

There is an exploration-exploitation trade-off, as in every RL strategy. If one does not use ```epsilon```, one would only focus on the transitions ```(state, action)``` that yields the best result at a time ```t```, and would never explore. It could for instance not notice there is even better transitions, because it would never explore them.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3)) + 128
        b[self.board > 0,0] = 256
        b[self.board < 0,2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[-2:,:] = -1

        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size-3:
                self.y = self.y-1
            else:
                self.y = self.y+1
        elif action == 3:
            if self.y == 2:
                self.y = self.y+1
            else:
                self.y = self.y-1
        else:
            raise RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[-2:,:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` is a 2D array containing the reward on each cell. Hence, everytime the rat goes to a cell ```(i,j)```, the corresponding reward ```board[i,j]``` is returned and this value is set to zero because the rat has collected the cheese or the poison.


The array ```position``` is a 2D array containing 1 if the rat is occupying the cell, -1 is the cell is a well and 0 otherwise.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # We reset the env
        state = env.reset()
        win, lose = 0, 0
        game_over = False

        while not game_over:
            # The agent choose an action
            action = agent.act(state, train=False)

            # We sample the reaction of the env
            prev_state = state
            state, reward, game_over = env.act(action)

            # And we update the counts
            if reward > 0:
                win = win + reward
            else:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/13.0. Average score (-3.5)
Win/lose count 12.5/19.0. Average score (-5.0)
Win/lose count 11.0/13.0. Average score (-4.0)
Win/lose count 9.5/16.0. Average score (-4.625)
Win/lose count 6.5/16.0. Average score (-5.6)
Win/lose count 7.0/20.0. Average score (-6.833333333333333)
Win/lose count 6.5/9.0. Average score (-6.214285714285714)
Win/lose count 9.0/19.0. Average score (-6.6875)
Win/lose count 12.0/23.0. Average score (-7.166666666666667)
Win/lose count 11.0/20.0. Average score (-7.35)
Win/lose count 8.5/14.0. Average score (-7.181818181818182)
Win/lose count 7.5/15.0. Average score (-7.208333333333333)
Win/lose count 9.0/12.0. Average score (-6.884615384615385)
Win/lose count 7.0/16.0. Average score (-7.035714285714286)
Win/lose count 10.0/12.0. Average score (-6.7)
Win/lose count 10.5/10.0. Average score (-6.25)
Win/lose count 5.5/18.0. Average score (-6.617647058823529)
Win/lose count 9.0/9.0. Average score (-6.25)
Win/lose count 10.0/15.0. Average score (-6.1842

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




<hr>

**Demonstration 1**

\begin{align*}
    Q^\pi(s,a) &= \mathbb{E}_{p^{\pi}}\left[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t}) \mid s_{0}=s,a_{0}=a\right] \\
    &= r(s,a) + \mathbb{E}_{p^{\pi}}\left[ \sum_{t=1}^{+\infty} \gamma^t r(s_t, a_t) \mid s_0=s, a_0=a\right] \\
    &= r(s,a) + \gamma \sum_{s'} \mathbb{E}_{p^{\pi}} \left[ \sum_{t=1}^{+\infty} \gamma^{t-1} r(s_t, a_t) \mid s_1 = s' \right] p(s' \mid s, a) \tag{1} \\
    &= r(s,a) + \gamma \sum_{s'} \sum_{a'} \mathbb{E}_{p^{\pi}} \left[ \sum_{t=1}^{+\infty} \gamma^{t-1} r(s_t, a_t) \mid s_1 = s', a_1 = a' \right] \pi(a' \mid s') p(s' \mid s,a) \tag{2} \\
    &= r(s,a) + \gamma \sum_{s'} \sum_{a'} Q^{\pi}(s',a') p^{\pi}(s', a' \mid s,a)\tag{3} \\
    &= \mathbb{E}_{(s',a') \sim p^{\pi}(.,.\mid s,a)}\left[ r(s,a) + \gamma Q^{\pi}(s',a') \right]
\end{align*}

$(1)$ is because we do a conditioning on the next state and we use the Markov property. \\
$(2)$ is because we do a conditioning on the next state, we use the definition of the conditioning to rewrite the probability, and we use the Markov property to remove the depency of the past states in $\pi$. Note that actions are selected according to $\pi$ when considering $Q^{\pi}$, hence the term in $\pi$. \\
$(3)$ is because we can operate a change of time scale $t' = t-1$ and translate every indice. Starting from $0$ and doing an infinite amount of operations is the same as starting from $1$ and doing that same infinite amount of operations. It's just a question of indices and it won't change the final reward.

<br>
<hr>

**Demonstration 2**

Let us now introduce a few notations, for $f : \mathcal{X} \times \mathcal{A} \to \mathbb{R}$ (where $\mathcal{A}$ is the action space and $\mathcal{X}$ is the state space), and for $(s,a) \in \mathcal{X} \times \mathcal{A}$:
$$\mathcal{T}^{\pi} f(s,a) = r(s,a) + \gamma \sum_{s' \in \mathcal{X}} \sum_{a' \in \mathcal{A}} f(s', a') \pi(a' \mid s') p(s' \mid s,a)$$
$$\mathcal{T} f(s,a) = r(s,a) + \gamma \sum_{s' \in \mathcal{X}} \max_{a' \in \mathcal{A}} f(s',a') p(s' \mid s,a)$$

**Part 1**

Let's demonstrate $\mathcal{T}^{\pi} f \leq \mathcal{T} f$ **(4)**:

\begin{align*}
    \mathcal{T}^{\pi}f(s,a) &= r(s,a) + \gamma \sum_{s'} \sum_{a'} f(s',a') \pi(a' \mid s') p(s' \mid s,a) \\
    &\leq r(s,a) + \gamma \sum_{s'} \sum_{a'} \max_{a''} f(s', a'') \pi(a' \mid s') p(s' \mid s,a) \\
    &\leq r(s,a) + \gamma \sum_{s'} \max_{a'} f(s',a') p(s' \mid s,a) \\
    &= \mathcal{T} f(s,a)
\end{align*}
By using the fact that $\pi \geq 0$ and $p \geq 0$.

**Part 2**

Let us now show that for any policy $\pi$, if $g \leq f$, then $\mathcal{T}^{\pi} f \leq \mathcal{T}^{\pi} g$ **(5)**:
$$\mathcal{T}^{\pi} f = r(s,a) + \gamma \sum_{s'}\sum_{a'} f(s',a') \pi(a' \mid s') p(s' \mid s,a) \leq r(s,a) + \gamma \sum_{s'}\sum_{a'} g(s',a') \pi(a' \mid s') p(s' \mid s,a) = \mathcal{T}^{\pi} g$$
Because of the positiviy of $\pi(a' \mid s') p(s' \mid s,a)$ for all $s,a,s',a'$.

**Part 3**

Finally, let us show that $\left(\mathcal{T}^{\pi}\right)^n f \xrightarrow[n \to +\infty]{} Q^{\pi}$ **(6)**:

The operator $\mathcal{T}^{\pi}$ is contractive:
\begin{align*}
    \left|\mathcal{T}^{\pi} f(s,a) - \mathcal{T}^{\pi} g(s,a)\right| &= \gamma \left|\sum_{s'}\sum_{a'} (f-g) \pi(a' \mid s') p(s' \mid s,a) \right| \\
    &\leq \gamma \left\|f-g\right\|_{\infty} 
\end{align*}
Hence:
$$\left\|\mathcal{T}^{\pi} f - \mathcal{T}^{\pi} g\right\|_{\infty} \leq \gamma \left\|f-g\right\|_{\infty}$$

Hence as we have $\mathcal{T}^{\pi} Q^{\pi} = Q^{\pi}$, and we suppose $\gamma < 1$:
$$\left\|\left(\mathcal{T}^{\pi}\right)^n f - Q^{\pi}\right\|_{\infty} \leq \gamma^n \left\|f - Q^{\pi}\right\|_{\infty} \xrightarrow[n \to +\infty]{} 0$$

**Part 4**

Now, let us define the deterministic policy $\hat{\pi}$:
$$\hat{\pi}(s) = \text{arg}\max_{a \in \mathcal{A}} Q^*(s,a) = \text{arg}\max_{a\in \mathcal{A}} Q^{\pi^*}(s,a)$$

Now, we have by **(4)**:
$$Q^{\pi^*} = \mathcal{T}^{\pi^*} Q^{\pi^*} \leq \mathcal{T} Q^{\pi^*}$$ 
And:
$$\mathcal{T} Q^{\pi^*} = r(s,a)+ \gamma \sum_{s'} \max_{a'} Q^{\pi^*}(s',a') p(s' \mid s,a) = r(s,a) + \gamma \sum_{s'} \sum_{a'} Q^{\pi^*} (s',a') \hat{\pi}(a' \mid s') p(s' \mid s,a) = \mathcal{T}^{\hat{\pi}} Q^{\pi^*}$$
Hence:
$$Q^{\pi^*} \leq \mathcal{T}^{\hat{\pi}} Q^{\pi^*}$$
By using iteratively **(5)** with $f = Q^{\pi^*}$ and $g = \mathcal{T}^{\hat{\pi}} Q^{\pi^*}$, we get:
$$Q^{\pi^*} \leq \left(\mathcal{T}^{\hat{\pi}}\right)^n Q^{\pi^*}$$
And by **(6)**, we get by making $n$ go to $+\infty$:
$$Q^{\pi^*} \leq Q^{\hat{\pi}}$$
By maximality of $\pi^*$, we get:
$$Q^{\pi^*} = Q^* = Q^{\hat{\pi}}$$
And:
\begin{align*}
Q^*(s,a) &= r(s,a) + \gamma \sum_{s'} \sum_{a'} Q^*(s',a') \hat{\pi}(s',a') p(s' \mid s,a) \\
&= r(s,a) + \gamma \sum_{s'} \max_{a'} Q^*(s',a') p(s' \mid s,a)
\end{align*}
By definition of $\hat{\pi}$, which ends the proof.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model.predict(s[None]).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i] = s
            target_q[i] = self.model.predict(s[None])

            if game_over:
                pass
            else:
                target_q[i,a] = r + self.discount * self.model.predict(n_s[None]).max()
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)

        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

from keras.layers import Flatten

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dense(4))
    
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(keras.optimizers.Adam(lr=1e-3), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/020 | Loss 0.0013 | Win/lose count 4.5/8.0 (-3.5)
Epoch 001/020 | Loss 0.0021 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/020 | Loss 0.0020 | Win/lose count 5.5/6.0 (-0.5)
Epoch 003/020 | Loss 0.0023 | Win/lose count 2.5/6.0 (-3.5)
Epoch 004/020 | Loss 0.0011 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/020 | Loss 0.0004 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/020 | Loss 0.0007 | Win/lose count 10.5/5.0 (5.5)
Epoch 007/020 | Loss 0.0007 | Win/lose count 7.0/1.0 (6.0)
Epoch 008/020 | Loss 0.0009 | Win/lose count 5.0/1.0 (4.0)
Epoch 009/020 | Loss 0.0013 | Win/lose count 8.5/1.0 (7.5)
Epoch 010/020 | Loss 0.0020 | Win/lose count 7.0/2.0 (5.0)
Epoch 011/020 | Loss 0.0011 | Win/lose count 4.0/1.0 (3.0)
Epoch 012/020 | Loss 0.0004 | Win/lose count 6.5/1.0 (5.5)
Epoch 013/020 | Loss 0.0003 | Win/lose count 7.5/1.0 (6.5)
Epoch 014/020 | Loss 0.0009 | Win/lose count 10.0/3.0 (7.0)
Epoch 015/020 | Loss 0.0006 | Win/lose count 5.5/2.0 (3.5)
Epoch 016/020 | Loss 0.0007 | Win/lose

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(5, 3, padding="same", activation="relu"))
        model.add(Conv2D(5, 3, padding="same", activation="relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(keras.optimizers.Adam(lr=1e-2), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0009 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/020 | Loss 0.0015 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/020 | Loss 0.0009 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/020 | Loss 0.0010 | Win/lose count 11.0/4.0 (7.0)
Epoch 004/020 | Loss 0.0008 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/020 | Loss 0.0020 | Win/lose count 8.0/6.0 (2.0)
Epoch 006/020 | Loss 0.0015 | Win/lose count 11.5/1.0 (10.5)
Epoch 007/020 | Loss 0.0009 | Win/lose count 5.0/5.0 (0.0)
Epoch 008/020 | Loss 0.0010 | Win/lose count 13.0/3.0 (10.0)
Epoch 009/020 | Loss 0.0007 | Win/lose count 9.5/4.0 (5.5)
Epoch 010/020 | Loss 0.0006 | Win/lose count 7.5/1.0 (6.5)
Epoch 011/020 | Loss 0.0017 | Win/lose count 11.5/3.0 (8.5)
Epoch 012/020 | Loss 0.0052 | Win/lose count 7.5/3.0 (4.5)
Epoch 013/020 | Loss 0.0060 | Win/lose count 13.5/1.0 (12.5)
Epoch 014/020 | Loss 0.0004 | Win/lose count 7.5/2.0 (5.5)
Epoch 015/020 | Loss 0.0066 | Win/lose count 15.5/3.0 (12.5)
Epoch 016/020 | Loss 0.0005 | Win/lose count 6

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.6) # 0.3 initially
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0.0. Average score (1.0)
Win/lose count 12.0/0.0. Average score (6.5)
Win/lose count 0.5/0.0. Average score (4.5)
Win/lose count 13.0/1.0. Average score (6.375)
Win/lose count 1.5/0.0. Average score (5.4)
Win/lose count 9.0/0.0. Average score (6.0)
Win/lose count 4.5/0.0. Average score (5.785714285714286)
Win/lose count 9.5/0.0. Average score (6.25)
Win/lose count 9.5/1.0. Average score (6.5)
Win/lose count 3.5/0.0. Average score (6.2)
Win/lose count 6.0/0.0. Average score (6.181818181818182)
Win/lose count 5.5/1.0. Average score (6.041666666666667)
Win/lose count 2.0/0.0. Average score (5.730769230769231)
Win/lose count 4.5/0.0. Average score (5.642857142857143)
Win/lose count 9.5/0.0. Average score (5.9)
Win/lose count 0.5/0.0. Average score (5.5625)
Win/lose count 6.0/0.0. Average score (5.588235294117647)
Win/lose count 4.0/1.0. Average score (5.444444444444445)
Win/lose count 4.0/0.0. Average score (5.368421052631579)
Win/lose count 9.5/0.0. Aver

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

One can notice that the CNN approach seem to barely never get any poison, and the FC approach seems to get it more frequently, but it's rare. Both approach seem to be stuck when reaching a certain configuration of the state, they don't do any exploration.

When one sets the temperature to be very low (0.05), one notice that the score of both approaches are really low, since they get stuck rapidly in a configuration where they loop and stop exploring.

When one sets the temperature to be quite high (0.6), one notice that the average score gets higher (3.95 for the CNN approach, 0.55 for the FC approach). It might be explained by the fact that since there is way more things to eat, they more frequently reach a configuration where there is food nearby, and hence they go to that direction, which simulates some kind of exploration.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
##########################
# train_explore function #
##########################

def train_explore(agent, env, epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    # We set the agent so he explores a lot at the beginning
    agent.set_epsilon(0.5)

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        # We change the exploration rate of our agent
        agent.set_epsilon(0.5 * (1/np.sqrt(epoch+1)))

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

##############################
# EnvironmentExploring class #
##############################

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # History of positions
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
    
    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0,0] = 256
        b[self.board < 0,2] = 256
        b[self.x,self.y,:] = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:] = 0
        b[:,:2,:] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[-2:,:] = -1

        self.position[self.x, self.y] = 1

        # Update the position
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size-3:
                self.y = self.y-1
            else:
                self.y = self.y+1
        elif action == 3:
            if self.y == 2:
                self.y = self.y+1
            else:
                self.y = self.y-1
        else:
            raise RuntimeError('Error: action not recognized')

        # Update time and check if finished
        self.t = self.t + 1
        game_over = self.t > self.max_time

        # Get reward
        reward = self.board[self.x, self.y]
        if train:
            reward -= self.malus_position[self.x, self.y]
        
        # A "plus" here helped, it helps prevent loops
        self.malus_position[self.x, self.y] += 0.1

        # Update the board
        self.board[self.x, self.y] = 0

        # Create new state
        state = np.concatenate((self.board[:,:,None],
                                self.position[:,:,None],
                                self.malus_position[:,:,None]), axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        # Generate initial position
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # Compute the position of the poison and cheese
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        malus[bonus>0] = 0

        # Define new board and timeline
        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[-2:,:] = -1
        self.board[self.x,self.y] = 0

        # Reset the visited cells
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # Reset the timer
        self.t = 0

        # Return new state
        state = np.concatenate((self.board[:,:,None],
                                self.position[:,:,None],
                                self.malus_position[:,:,None]),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0203 | Win/lose count 9.5/51.40000000000003 (-41.90000000000003)
Epoch 001/020 | Loss 0.0138 | Win/lose count 18.0/47.60000000000001 (-29.60000000000001)
Epoch 002/020 | Loss 0.0083 | Win/lose count 16.0/29.50000000000002 (-13.500000000000021)
Epoch 003/020 | Loss 0.0302 | Win/lose count 17.0/30.499999999999993 (-13.499999999999993)
Epoch 004/020 | Loss 0.0143 | Win/lose count 17.0/26.50000000000003 (-9.500000000000028)
Epoch 005/020 | Loss 0.0163 | Win/lose count 23.0/24.700000000000006 (-1.7000000000000064)
Epoch 006/020 | Loss 0.0087 | Win/lose count 13.0/41.100000000000016 (-28.100000000000016)
Epoch 007/020 | Loss 0.0125 | Win/lose count 18.5/19.199999999999996 (-0.6999999999999957)
Epoch 008/020 | Loss 0.0121 | Win/lose count 17.5/34.70000000000004 (-17.20000000000004)
Epoch 009/020 | Loss 0.0121 | Win/lose count 19.0/26.600000000000023 (-7.600000000000023)
Epoch 010/020 | Loss 0.0100 | Win/lose count 23.0/23.300000000000015 (-0.3000000000000149)
Epoch 011/

In [20]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 19.0/10.0. Average score (9.0)
Win/lose count 21.5/10.0. Average score (10.25)
Win/lose count 21.5/8.0. Average score (11.333333333333334)
Win/lose count 27.0/8.0. Average score (13.25)
Win/lose count 19.0/7.0. Average score (13.0)
Win/lose count 19.5/12.0. Average score (12.083333333333334)
Win/lose count 21.5/6.0. Average score (12.571428571428571)
Win/lose count 26.0/7.0. Average score (13.375)
Win/lose count 26.5/3.0. Average score (14.5)
Win/lose count 14.0/10.0. Average score (13.45)
Win/lose count 25.0/9.0. Average score (13.681818181818182)
Win/lose count 18.5/4.0. Average score (13.75)
Win/lose count 13.5/7.0. Average score (13.192307692307692)
Win/lose count 21.0/6.0. Average score (13.321428571428571)
Win/lose count 22.5/15.0. Average score (12.933333333333334)
Win/lose count 23.0/6.0. Average score (13.1875)
Win/lose count 24.0/8.0. Average score (13.352941176470589)
Win/lose count 28.5/8.0. Average score (13.75)
Win/lose count 27.0/6.0. Average score (14.131

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
# Agent definition
class MimickAgent(Agent):
    def __init__(self, expert_model, grid_size, epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(MimickAgent, self).__init__(epsilon = epsilon)

        # Expert model
        self.expert_model = expert_model

        # Set the exploration rate to 0
        self.set_epsilon(0)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s): # Trajectories according to the expert model
        return self.expert_model.predict(s[None]).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()

            # What would the expert model do?
            input_states[i] = s
            target_q[i] = self.model.predict(s[None])

            if game_over:
                pass
            else:
                target_q[i,a] = r + self.discount * self.model.predict(n_s[None]).max()
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l
    
    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)

        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self, name_weights='model.h5', name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class MimickAgent_CNN(MimickAgent):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(MimickAgent_CNN, self).__init__(*args, **kwargs)
        
        model = Sequential()
        model.add(Conv2D(5, 3, padding="same", activation="relu"))
        model.add(Conv2D(5, 3, padding="same", activation="relu"))
        model.add(Flatten())
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(keras.optimizers.Adam(lr=1e-2), "mse")
        self.model = model

In [22]:
# Training
env_mimick = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_mimick = MimickAgent_CNN(agent.model, size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent_mimick, env_mimick, epochs_train, prefix='cnn_train_mimick')
HTML(display_videos('cnn_train_mimick10.mp4'))

Epoch 000/020 | Loss 0.0124 | Win/lose count 20.0/33.70000000000003 (-13.700000000000031)
Epoch 001/020 | Loss 0.0058 | Win/lose count 28.5/21.399999999999995 (7.100000000000005)
Epoch 002/020 | Loss 0.0092 | Win/lose count 22.5/17.999999999999996 (4.5000000000000036)
Epoch 003/020 | Loss 0.0036 | Win/lose count 22.5/25.700000000000014 (-3.2000000000000135)
Epoch 004/020 | Loss 0.0036 | Win/lose count 25.5/18.799999999999986 (6.7000000000000135)
Epoch 005/020 | Loss 0.0053 | Win/lose count 26.5/17.199999999999985 (9.300000000000015)
Epoch 006/020 | Loss 0.0089 | Win/lose count 19.0/19.899999999999995 (-0.899999999999995)
Epoch 007/020 | Loss 0.0039 | Win/lose count 26.0/19.40000000000001 (6.599999999999991)
Epoch 008/020 | Loss 0.0050 | Win/lose count 21.5/30.000000000000004 (-8.500000000000004)
Epoch 009/020 | Loss 0.0110 | Win/lose count 23.5/18.399999999999995 (5.100000000000005)
Epoch 010/020 | Loss 0.0060 | Win/lose count 22.5/23.999999999999986 (-1.4999999999999858)
Epoch 011/020

In [23]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_mimick')
HTML(display_videos('cnn_test_mimick10.mp4'))

Win/lose count 21.0/6.0. Average score (15.0)
Win/lose count 22.5/5.0. Average score (16.25)
Win/lose count 21.0/9.0. Average score (14.833333333333334)
Win/lose count 21.0/8.0. Average score (14.375)
Win/lose count 19.0/4.0. Average score (14.5)
Win/lose count 20.5/5.0. Average score (14.666666666666666)
Win/lose count 26.5/8.0. Average score (15.214285714285714)
Win/lose count 19.5/5.0. Average score (15.125)
Win/lose count 26.5/5.0. Average score (15.833333333333334)
Win/lose count 16.0/6.0. Average score (15.25)
Win/lose count 22.0/2.0. Average score (15.681818181818182)
Win/lose count 20.0/4.0. Average score (15.708333333333334)
Win/lose count 29.0/4.0. Average score (16.423076923076923)
Win/lose count 25.5/7.0. Average score (16.571428571428573)
Win/lose count 22.0/9.0. Average score (16.333333333333332)
Win/lose count 23.5/4.0. Average score (16.53125)
Win/lose count 20.0/6.0. Average score (16.38235294117647)
Win/lose count 16.0/5.0. Average score (16.083333333333332)
Win/lose 

The model that mimick the behaviour of the expert model performs better than the expert model. It can be explained by the fact that it is trained only on the optimal transitions and its training is not "altered" by the random transitions that sometimes yield bad results.

***